# Article 9


## Objectif:

Cartographier les acteurs de la proposition de loi de l'article 9

### Découverte du dataset

Un fichier json avec les donnees relatives a l'article 9 est disponible
article9.json

    ! Le détail des votes par utilisateurs n'est disponible que dans le fichier participants

Ce fichier représente les données sous forme de dictionnaire: accessible par clé,valeur nous l'avons 
stocké dans la variable **article9**

La première démarche pour connaîre les données et d'afficher les clés disponibles du dictionnaires

Affichons les clés du dictionnaire:

In [37]:
import json, pprint
#afficher les données plus zouliement avec indentation
pp = pprint.PrettyPrinter(indent=4, depth=4)
with open("article9.json", "r") as f:
    article9 = json.load(f)
    pp.pprint(article9.keys())

[   u'subtitle',
    u'cat_id',
    u'updated_at',
    u'sources',
    u'arguments_count',
    u'versions_count',
    u'votes_total',
    u'author',
    u'article_link',
    u'arguments',
    u'answer',
    u'body',
    u'ranking',
    u'sources_count',
    u'votes_ok',
    u'versions',
    u'article_id',
    u'body_anchors',
    u'created_at',
    u'title',
    u'votes_mitige',
    u'body_links',
    u'votes_nok']


* Statistiques descriptives de l'article 9:

Nous allons charger les informations disponibles dans une **variable** appelée re_article9
qui va contenir les differents champs statistiques disponibles en les réorganisant légèrement pour une plus grande facilité de manipulation. 
Ici nous filtrons les clés du dictionnaires en fonction de leur nom que nous connaissons grace à la manipulation 
ci-dessus.

On va réorganiser de la facon suivante
[  # l'identifiant de l'article est l'url
    #on le transfrome en id
    u'article_link',
    #date
    u'created_at',
    u'updated_at',
    #stats
    u'arguments_count',
    u'versions_count',
    u'votes_total',
    u'sources_count',
    
    # repartition des votes
    u'votes_ok',
    u'votes_mitige'
    u'votes_nok'
    #détails participation
    u'versions',
    u'sources',
    u'arguments',
    u'votes',
    ]

In [38]:
import json, pprint
ref_article9 = {"date": {}, "stats": {"votes":{}}, "details": {"votes":[]}, "votes":{}}
#pour afficher les données joliement
pp = pprint.PrettyPrinter(indent=4)
with open("article9.json", "r") as f:
    data = json.load(f)
    for k,v in data.items():
        
        if k == "article_link":
            ref_article9["id"] = v
        #match with date elemnts
        if k.endswith("at"):
            ref_article9["date"][k] = v
        elif k.endswith("count"):
            ref_article9["stats"][k.replace("count", "nb")] = v
        elif k.endswith("total"):
            ref_article9["stats"][k.replace("total", "nb")] = v
        elif k.startswith("votes"):
            ref_article9["stats"]["votes"][k.replace("votes_","")] = v
            
        else:
            if k in ["versions", "sources", "arguments"]:
                ref_article9["details"][k] = v
            pass
        #stats = {k:v for k,v in data.items() if k.endswith("at") and if  and if k.endswith("total") and if k.startswith("votes")}
pp.pprint(ref_article9["stats"])
#resultats des statistiques de l'article

{   u'arguments_nb': 132,
    u'sources_nb': 22,
    u'versions_nb': 108,
    'votes': {   u'mitige': 282, u'nok': 1732, u'ok': 1320},
    u'votes_nb': 3334}


L'article 9 compte donc 132 arguments,22 sources, 3334 votes et 108 versions.
Dans "details":
* Chaque argument contient le nombre de votes concernant l'argument
* Chaque argument contient le nombre de votes concernant la version


Le nombre de votes total correspond aux votes sur l'article original et le nombre d'arguments aussi
et non pas sur l'ensemble des versions de l'article


###Les modifications de l'article
Les amemendements ou modification de l'article sont disponible 
désormais dans ref_article9["details"]["versions"] sous forme de liste de dictionnaire
Explorons donc ensemble les différentes valeurs exploitables

In [39]:
versions = {}
for version in ref_article9["details"]["versions"]:
    print pp.pprint(version.keys())
    break

[   u'comment',
    u'author',
    u'arguments_no_count',
    u'created_at',
    u'after',
    u'updated_at',
    u'id',
    u'votes_mitige',
    u'arguments_yes_count',
    u'link',
    u'arguments_count',
    u'before',
    u'title',
    u'votes_total',
    u'article_id',
    u'slug',
    u'votes_nok']
None


Nous allons réorganiser le contenu des references de versions
pour rappatrier ensuite les votes qui contient les données détaillées 

[   #u'slug',
    #u'title'
    #date
    u'created_at',
    u'updated_at',
    u'author',
    #stats
        #arguments
        u'arguments_no_count',
        u'arguments_yes_count',
        u'arguments_count',
        #votes
        u'votes_total',
        u'votes_mitige',
        u'votes_nok',
        u'votes_ok'
    ]
dans un dictionnaire
dont l'id unique sera le slug
versions[slug] = {"title":"", "date":{}, "author": "", "stats":{"votes":{}, "arguments":{}}



In [47]:
versions = {}
for vers in ref_article9["details"]["versions"]:
    versions[vers["slug"]] = {"title":vers["title"], 
                              "date":{}, 
                              "author": vers["author"], 
                              "stats":{"arguments":{},"votes":{}},
                              "arguments":[],
                              "votes":[],
                             }
    for k,v in vers.items():
        if k.endswith("at"):
            versions[vers["slug"]]["date"][k] = v
        elif k.startswith("votes"):
            versions[vers["slug"]]["stats"]["votes"][k] = v
        elif k.startswith("arguments"):
            versions[vers["slug"]]["stats"]["arguments"][k] = v
        else:
            pass
versions["original"] = ref_article9
versions["original"]["author"] = "gouvernement"
versions["original"]["votes"] = []
#versions["original"]["arguments"] = []
#On a bien 108 versions dans ce dictionnaire + 1 version originale
#print len(versions.keys())
#print versions["attention-a-la-qualite-et-a-la-securite-des-textes-scientifiques-qui-doivent-etre-valides-en-amont-promus-et-enrichis-ensuite-par-le-biais-de-bases-de-donnees"]
print versions["original"]

{'votes': [], 'stats': {u'versions_nb': 108, 'votes': {u'mitige': 282, u'ok': 1320, u'nok': 1732}, u'arguments_nb': 132, u'votes_nb': 3334, u'sources_nb': 22}, 'author': 'gouvernement', 'details': {u'sources': [{u'body': u"Site web illustrant l'opposition entre certains chercheurs et un \xe9diteur (ici Elsevier).", u'category': u'Site internet', u'title': u'The Cost of Knowledge', u'source_link': u'http://thecostofknowledge.com/', u'created_at': u'2015-10-17T22:33:33+0200', u'author': u'vincentreverdy', u'updated_at': u'2015-10-17T22:33:33+0200', u'link': u'http://thecostofknowledge.com/#source-380', u'votes_count': u'1', u'article_id': 69, u'id': u'380'}, {u'body': u'La LERU (Ligue des Universit\xe9s de Recherche Europ\xe9ennes) demande une attitude coh\xe9rente sur ce sujet au niveau europ\xe9en', u'category': u'Rapport / \xc9tude', u'title': u"Recommandations de la LERU sur l'Open access", u'source_link': u'http://www.leru.org/files/general/LERU%20Statement%20Moving%20Forwards%20on%

In [41]:
arguments = {}
for arg in ref_article9["details"]["arguments"]:
    #ici l'id unique est l'id
    #"arg-"+id
    arguments[arg["id"]] = {"date":{}, "stats":{"votes":{}}}
    for k, v in arg.items():
        if k.endswith("at"):
            arguments[arg["id"]]["date"][k] = v
    arguments[arg["id"]]["stats"]["votes"]["total"] = v
            

#on a bien les 132 arguments annoncés pour la version original 
print(len(arguments))

print arguments["3923"]

132
{'date': {u'created_at': u'2015-10-14T15:51:34+0200', u'updated_at': u'2015-10-14T15:52:07+0200'}, 'stats': {'votes': {'total': u'3923'}}}


Dans un deuxième temps nous allons croiser ces données avec ceux des votes
pour trouver qui a voté pour quelle modif ou quelle version
Pour cela il nous faut trouver tous les votes pour l'article9
Le détail des votes est disponible dans le fichier participants
qui liste sous forme de dictionnaire de participants toutes l'activité du participant


### Qui a voté quoi?
On va passer en revue tous les votes des participants et trouver ceux qui correspondent à l'article qui nous intéresse
en filtrant sur le nom de l'article contenu dans le lien
et construire ainsi un référentiel de votes


In [48]:
url = "section-2-travaux-de-recherche-et-de-statistique"
art = "article-9-acces-aux-travaux-de-la-recherche-financee-par-des-fonds-publics"
#article9 = ref_article9
#raccourci
#votes = article9["details"]["votes"] = []
#arguments = article9["details"]["arguments"]
#version = article9["details"]["versions"]

with open("participants.json", "r") as f:
    data = json.load(f)
    participants = data["participants"]


#rappel versions contient toutes les versions de l'article
for slug, version in versions.items():
    print version["votes"]
    break
#versions= []
votes_original = []
for user in participants:
    username = user.keys()[0]
    user_votes =  user[username]["votes"]
    for vote in user_votes:
        url = vote["link"]
        if not art in url:
            continue
        else:
            vote["user"] = username
            if "version" in url:
                #ici il s'agit d'une version mais laquelle?
                #avant vérifier que ce n'est pas un vote sur un arg, source
                if not "#arg" in url and not "#source" in url:
                    slug = url.split("/")[-1]
                    versions[slug]["votes"].append(vote)
                    continue
                else:
                    continue
            else:
                if not "#arg" in url and not "#source" in url:
                    slug = url.split("/")[-1]
                    #print slug
                    versions["original"]["votes"].append(vote)
                    #votes_original.append(vote)
                    continue
                else:
                    continue

print len(versions["original"]["votes"])

'''
for participation in participants:
    for user, part in participation.items():
        for vote in part["votes"]:
            url = vote["link"]
            #filtrer les articles
            if art in url:
                votant = {"user":user, "date": vote["date"], "opinion": vote["opinion"], "id":vote["id"], "url":url}
                if "version" in url:
                    slug = url.split("/")[-1]
                    
                    if "arg" in url:
                        #ici il s'agit d'un vote sur l'argument de cette version
                        arg_id = slug.split("#arg-")[-1]
                        versions[slug]["arguments"].append(votant)
                        #print arg_id
                        continue
                    elif "source" in url:
                        continue
                    else:
                        versions[slug]["votes"].append(votant)
                        continue
                else:
                    if "#arg" in url:
                        #un vote sur l'argument
                        #versions["original"]["votes"].append(votant)
                        continue
                    elif "#source" in url:
                        #versions["original"]["details"]["sources_votes"].append(votant)
                        continue
                    else:
                        #ici il s'agit d'un vote sur l'article original
                        #ref_article9["details"]["votes"].append(votant)
                        versions["original"]["votes"].append(votant)
                        continue
'''
#on vérifie les votes
#votants
#print len(ref_article9["details"]["votes"])
#votes comptabilisés dans l'article
#print ref_article9["stats"]["votes_nb"]


[]
3334


'\nfor participation in participants:\n    for user, part in participation.items():\n        for vote in part["votes"]:\n            url = vote["link"]\n            #filtrer les articles\n            if art in url:\n                votant = {"user":user, "date": vote["date"], "opinion": vote["opinion"], "id":vote["id"], "url":url}\n                if "version" in url:\n                    slug = url.split("/")[-1]\n                    \n                    if "arg" in url:\n                        #ici il s\'agit d\'un vote sur l\'argument de cette version\n                        arg_id = slug.split("#arg-")[-1]\n                        versions[slug]["arguments"].append(votant)\n                        #print arg_id\n                        continue\n                    elif "source" in url:\n                        continue\n                    else:\n                        versions[slug]["votes"].append(votant)\n                        continue\n                else:\n            

In [43]:
### Qui a voté pour qui? Pour quelle version?


In [49]:
#auteur, votant, vote, slug
final_dataset = []
for slug, version in versions.items():
    
    auteur = version["author"]
    for vote in version["votes"]:
        final_dataset.append([vote["user"],auteur,vote["opinion"], slug])
        

#On ajoute les votants à l'article original
#for votant in ref_article9["details"]["votes"]:
    #print votant
    #final_dataset.append([ vote["user"],"gouvernement",vote["opinion"], "original"])
    


In [50]:
with open("electeur_auteur.csv", "w") as f:
    f.write("\t".join(["electeur","auteur", "vote", "version"])+'\n')
    for row in final_dataset:
        f.write("\t".join(row)+'\n')
        
    

## Détection des communautés
Mais avant nous allons faire des stats pour voir si tout est bien carré!
Combien de votes en tout

In [51]:
print len(final_dataset)

12851


On a 12851 contre 3334 votes sur la version original
Maintenant vote_pour, vote_contre, votes_mitigés

In [54]:
votes_pour = []
votes_contre = []
votes_neutre = []
for vote in final_dataset:
    electeur, auteur, opinion, version = vote
    if opinion == "1":
        votes_pour.append(vote)
    elif opinion == "-1":
        votes_contre.append(vote)
    else:
        votes_neutre.append(vote)
print "votes pour: %i , votes contre: %i votes neutres:%i" %(len(votes_pour),len(votes_contre), len(votes_neutre))

votes pour: 8926 , votes contre: 3425 votes neutres:500


Maintenant sans le gouvernement:

In [55]:
votes_pour = []
votes_contre = []
votes_neutre = []
for vote in final_dataset:
    electeur, auteur, opinion, version = vote
    if version == "original":
        continue
    if opinion == "1":
        votes_pour.append(vote)
    elif opinion == "-1":
        votes_contre.append(vote)
    else:
        votes_neutre.append(vote)
print "votes pour: %i , votes contre: %i votes neutres:%i" %(len(votes_pour),len(votes_contre), len(votes_neutre))

votes pour: 7606 , votes contre: 1693 votes neutres:218


In [56]:
#Si on veut comparer, le gouvernement a pris en compte seulement les votes sur l'original
print ref_article9["stats"]


{u'versions_nb': 108, 'votes': {u'mitige': 282, u'ok': 1320, u'nok': 1732}, u'arguments_nb': 132, u'votes_nb': 3334, u'sources_nb': 22}


In [65]:
#On vérifie que le croisement a bien marché "matching"
#entre les déclarations du nombre de votes
#et l'identité des votants (participants.json)
for slug, info in versions.items():
    #stats de votes
    try:
        if info["stats"]["votes"]["votes_total"] != len(info["votes"]):
            print slug, info["stats"], len(info["votes"])
    except KeyError:
        total = 0
        for n in ["ok", "nok", "mitige"]:
            total = total+info["stats"]["votes"][n]
        info["stats"]["votes"]["votes_total"] = total
        if total != len(info["votes"]):
            print slug, info["stats"], len(info["votes"])
#Bizarrerire votes_ok n'apparait pas

In [103]:
#Maintenant par version:
#Un fichier csv avec le nom de la version "slug", votes_pour, votes_contre, vote_neutre, total
#statistiques de votes par version
versions_votes_stats = []

#pour chaque version
for slug, info in versions.items():
    pour, contre, neutre, total = 0, 0, 0, len(versions[slug]["votes"])
    #dans  votes qui contient la liste des votants et leur opinion (0, 1, -1)
    # on recalcule les pour, contre, neutre
    for vote in versions[slug]["votes"]:
        if vote["opinion"] == "-1":
            contre = contre+1
        elif vote["opinion"] == "1":
            pour = pour+1
        else:
            neutre = neutre+1
    #on reecrit dans le dictionnaire des stats de votes propres a partir des données stockées
    versions[slug]["stats"]["votes"] = {"pour":pour, "contre": contre, "neutre": neutre,
                                      "total": total}
    #allez ca va marcher!
    print versions[slug]["stats"]["votes"]
    
    versions_votes_stats.append([slug, pour, contre, neutre, total])


{'neutre': 0, 'total': 7, 'contre': 3, 'pour': 4}
{'neutre': 6, 'total': 87, 'contre': 2, 'pour': 79}
{'neutre': 1, 'total': 24, 'contre': 7, 'pour': 16}
{'neutre': 8, 'total': 362, 'contre': 29, 'pour': 325}
{'neutre': 2, 'total': 35, 'contre': 18, 'pour': 15}
{'neutre': 18, 'total': 780, 'contre': 39, 'pour': 723}
{'neutre': 1, 'total': 7, 'contre': 0, 'pour': 6}
{'neutre': 2, 'total': 24, 'contre': 6, 'pour': 16}
{'neutre': 0, 'total': 0, 'contre': 0, 'pour': 0}
{'neutre': 1, 'total': 71, 'contre': 57, 'pour': 13}
{'neutre': 0, 'total': 29, 'contre': 0, 'pour': 29}
{'neutre': 0, 'total': 17, 'contre': 2, 'pour': 15}
{'neutre': 3, 'total': 74, 'contre': 4, 'pour': 67}
{'neutre': 0, 'total': 22, 'contre': 18, 'pour': 4}
{'neutre': 0, 'total': 1, 'contre': 0, 'pour': 1}
{'neutre': 3, 'total': 200, 'contre': 31, 'pour': 166}
{'neutre': 3, 'total': 32, 'contre': 20, 'pour': 9}
{'neutre': 1, 'total': 40, 'contre': 17, 'pour': 22}
{'neutre': 1, 'total': 6, 'contre': 0, 'pour': 5}
{'neutre'

In [104]:
#On a donc ub tableau statistique tout propre on va l'écrire dans un csv
with open("versions_stats.csv", "w") as f:
    
    f.write("\t".join(["version_slug","pour", "vote", "contre", "total"])+'\n')
    for row in versions_votes_stats:
        f.write("\t".join([str(n) for n in row])+'\n')


### Recap
On a 109 versions, 12851 votes
! Versions les plus votés = auteurs les plus élus
On a 102 auteurs uniques pour 109 versions
Quels sont les auteurs multirecidivistes

In [108]:
nb_auteurs = [v["author"] for v in versions.values()]
nb_auteurs_u = set(nb_auteurs)
auteurs_freq = [(n,nb_auteurs.count(n)) for n in nb_auteurs_u]
auteurs_multi = [n for n in auteurs_freq if n[1] >1]
print len(nb_auteurs), len(nb_auteurs_u), auteurs_multi

109 102 [(u'cbernault', 3), (u'pierrefrancoisjan', 2), (u'consortiumcouperin', 3), (u'samsond', 2), (u'melaniedulongderosnay', 2)]


In [123]:
electeurs = [n for n in final_dataset]
electeurs_u = [(n, electeurs.count(n)) for n in set(final_dataset)]
print len(electeurs_u)


TypeError: unhashable type: 'list'

In [ ]:
#Electeurs sont il similiares?


In [136]:
electeurs_d = {}

for n in list(set(electeurs)):
    electeurs_d[n] = []
    


for slug, version in versions.items():
    for vote in version["votes"]:
        auth = version["author"] 
        opinion = vote["author"]
        info = {"version": slug, "author": auth, "vote": opinion }
        electeurs_d[vote["user"]].append(info)

print electeurs_d["marindacos"]

TypeError: unhashable type: 'list'